# ChatGPT Inline Tips with simpleaichat

[simpleaichat](https://github.com/minimaxir/simpleaichat) allows the user more control over text inputs and output. One way to do it is to analyze user input for potential issues, and display a warning if there are unexpected issues with the input.

Inspired by Simon Willison's blog post [ChatGPT should include inline tips
](https://simonwillison.net/2023/May/30/chatgpt-inline-tips/).

**DISCLAIMER: This is far from a perfect solution, but a good proof-of-concept on how to use simpleaichat to address complex problems.**

In [1]:
!pip install -q simpleaichat

from simpleaichat import AIChat
from rich.console import Console
from getpass import getpass
import time

For the following cell, input your OpenAI API key when prompted. **It will not be saved to the notebook.**.

In [2]:
api_key = getpass("OpenAI Key: ")

OpenAI Key: ··········


Let's create an AI chat, and a Console for output to the notebook.

In [3]:
ai = AIChat(api_key=api_key, console=False, params={"temperature": 0.0})  # for reproducibility
console = Console(width=60, highlight=False)

First, we'll set a list of rules that we can display. These rules are adapted from real-world incidents and OpenAI's [usage policies](https://openai.com/policies/usage-policies).

In [4]:
tips = [
    "This ChatGPT model does not have access to the internet, and its training data cut-off is September 2021.",
    "ChatGPT should not be relied on for legal research of this nature, because it is very likely to invent realistic cases that do not exist.",
    "Medical and psychatric advice from ChatGPT should not be relied upon. Always consult a doctor.",
    "Tailored financial advice from ChatGPT should not be relied upon. Always consult a professional.",
    "ChatGPT is not liable for any illegal activies committed as the result of its responses."
]

When generating a response from ChatGPT, we'll first check the user input against a bespoke function in a different context to see if ChatGPT can detect one of the issues above. This is accomplished through a system prompt, and forcing the model to choose a number between `1` and `5`, or `0` if nothing is present.

The order of the tips above matches the order of the list. This is incidentially the same workflow as tools in simpleaichat.

In [5]:
tips_prompt = """From the list of topics below, reply ONLY with the number appropriate for describing the topic of the user's message. If none are, ONLY reply with "0".

1. Content after September 2021
2. Legal/Judicial Research
3. Medical/Psychatric Advice
4. Financial Advice
5. Illegal/Unethical Activies"""

In [6]:
# ensures tips output will always be an integer between 0 and 5 inclusive
params = {
    "temperature": 0.0,
    "max_tokens": 1,
    "logit_bias": {str(k): 100 for k in range(15, 15 + len(tips) + 1)}
}

# functional
ai.new_session(id="tips",
               api_key=api_key,
               system=tips_prompt,
               save_messages=False,
               params=params)

def check_user_input(message):
    tip_idx = ai(message, id="tips")
    if tip_idx == "0":  # no tip needed
        return
    else:
        tip = tips[int(tip_idx) - 1]
        console.print(f"⚠️ {tip}", style="bold")

Let's test it in a conversation!

In [11]:
while True:
    time.sleep(0.5)  # for Colab, to ensure input box appears
    try:
        user_input = console.input("[b]You:[/b] ").strip()
        if not user_input:
            break

        check_user_input(user_input)
        ai_response = ai(user_input)

        console.print(f"[b]ChatGPT[/b]: {ai_response}", style="bright_magenta")
    except KeyboardInterrupt:
        break

# ai.reset_session()

You:

Can you tell me more about Max v. Woolf?


⚠️ ChatGPT should not be relied on for legal research of this
nature, because it is very likely to invent realistic cases 
that do not exist.

ChatGPT: Max v. Woolf is a legal case that took place in the
United Kingdom in 2017. It involved a dispute between two 
authors, Max and Woolf, over the alleged plagiarism of Max's
work by Woolf. Max claimed that Woolf had copied significant
portions of his book without permission or attribution.

The case was heard in the High Court of Justice in London, 
where Max presented evidence of the similarities between his
work and Woolf's. Woolf denied the allegations and argued 
that any similarities were coincidental or the result of 
common themes in the genre.

After a lengthy trial, the judge ruled in favor of Max, 
finding that Woolf had indeed plagiarized his work. Woolf 
was ordered to pay damages to Max and to publicly 
acknowledge the plagiarism.

The case received significant media attention and sparked a 
wider discussion about the issue of plagiarism in the 
publishing industry. It also highlighted the importance of 
proper attribution and respect for intellectual property 
rights.

You:

...

ChatGPT: I'm sorry, but as an AI language model, I cannot 
assist you in committing plagiarism. Plagiarism is unethical
and illegal, and it can have serious consequences for both 
the person who commits it and the people whose work is 
plagiarized. It is important to always give proper credit to
the original source of any information or ideas that you use
in your own work. If you need help with writing or research,
there are many resources available that can assist you in 
doing so ethically and responsibly.

You:

ok bye then


ChatGPT: Goodbye! If you have any other questions or need 
assistance in the future, feel free to ask.

You:

## MIT License

Copyright (c) 2023 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
